In [2]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

import matplotlib.pyplot as plt
%matplotlib inline

## Tip Data Exploration
**Objective of this notebook**: Load Yelp Tip data into and explore it.

### 1. Load data as pandas df and convert it to .csv file

In [3]:
# convert json into csv file
tip_data = pd.read_json("archive/yelp_academic_dataset_tip.json", lines=True)

# save tip_data as csv
tip_data.to_csv("tip/yelp_academic_dataset_tip.csv", index=False)

# display
tip_data.head()

,business_id,compliment_count,date,text,user_id
0,UYX5zL_Xj9WEc_Wp-FrqHw,0,2013-11-26 18:20:08,Here for a quick mtg,hf27xTME3EiCp6NL6VtWZQ
1,Ch3HkwQYv1YKw_FO06vBWA,0,2014-06-15 22:26:45,Cucumber strawberry refresher,uEvusDwoSymbJJ0auR3muQ
2,rDoT-MgxGRiYqCmi0bG10g,0,2016-07-18 22:03:42,Very nice good service good food,AY-laIws3S7YXNl_f_D6rQ
3,OHXnDV01gLokiX1ELaQufA,0,2014-06-06 01:10:34,It's a small place. The staff is friendly.,Ue_7yUlkEbX4AhnYdUfL7g
4,GMrwDXRlAZU2zj5nH6l4vQ,0,2011-04-08 18:12:01,"8 sandwiches, $24 total...what a bargain!!! An...",LltbT_fUMqZ-ZJP-vJ84IQ


### 2. Load data into Spark

In [4]:
# start Spark session
spark = SparkSession.builder.appName('yelp_tip').getOrCreate()

# change configuration settings on Spark 
conf = spark.sparkContext._conf.setAll([('spark.executor.memory', '4g'), 
                                        ('spark.app.name', 'Spark Updated Conf'), 
                                        ('spark.executor.cores', '4'), 
                                        ('spark.cores.max', '4'), 
                                        ('spark.driver.memory','4g')])

# print spark configuration settings
spark.sparkContext.getConf().getAll()

[('spark.eventLog.enabled', 'true'),
 ('spark.yarn.jars',
  'local:/opt/cloudera/parcels/CDH-6.3.0-1.cdh6.3.0.p0.1279813/lib/spark/jars/*,local:/opt/cloudera/parcels/CDH-6.3.0-1.cdh6.3.0.p0.1279813/lib/spark/hive/*'),
 ('spark.yarn.appMasterEnv.MKL_NUM_THREADS', '1'),
 ('spark.sql.queryExecutionListeners',
  'com.cloudera.spark.lineage.NavigatorQueryListener'),
 ('spark.lineage.log.dir', '/var/log/spark/lineage'),
 ('spark.org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter.param.PROXY_HOSTS',
  'md01.rcc.local,md02.rcc.local'),
 ('spark.serializer', 'org.apache.spark.serializer.KryoSerializer'),
 ('spark.executorEnv.PYTHONPATH',
  '/opt/cloudera/parcels/CDH/lib/spark/python/lib/py4j-0.10.7-src.zip:/opt/cloudera/parcels/CDH/lib/spark/python/lib/pyspark.zip<CPS>/opt/cloudera/parcels/CDH-6.3.0-1.cdh6.3.0.p0.1279813/lib/spark/python/lib/py4j-0.10.7-src.zip<CPS>/opt/cloudera/parcels/CDH-6.3.0-1.cdh6.3.0.p0.1279813/lib/spark/python/lib/pyspark.zip'),
 ('spark.yarn.historyServer.addre

In [5]:
# load data into spark from hdfs
dat = spark.read.csv("/user/kleindiek/final_project/tip/yelp_academic_dataset_tip.csv", inferSchema=True, header=True)

In [6]:
# investigate data types
dat.dtypes

[('business_id', 'string'),
 ('compliment_count', 'string'),
 ('date', 'string'),
 ('text', 'string'),
 ('user_id', 'string')]

In [7]:
# investigate schema
dat.printSchema()

root
 |-- business_id: string (nullable = true)
 |-- compliment_count: string (nullable = true)
 |-- date: string (nullable = true)
 |-- text: string (nullable = true)
 |-- user_id: string (nullable = true)



In [8]:
# show first 10 rows
dat.show(10)

+--------------------+----------------+-------------------+--------------------+--------------------+
|         business_id|compliment_count|               date|                text|             user_id|
+--------------------+----------------+-------------------+--------------------+--------------------+
|UYX5zL_Xj9WEc_Wp-...|               0|2013-11-26 18:20:08|Here for a quick mtg|hf27xTME3EiCp6NL6...|
|Ch3HkwQYv1YKw_FO0...|               0|2014-06-15 22:26:45|Cucumber strawber...|uEvusDwoSymbJJ0au...|
|rDoT-MgxGRiYqCmi0...|               0|2016-07-18 22:03:42|Very nice good se...|AY-laIws3S7YXNl_f...|
|OHXnDV01gLokiX1EL...|               0|2014-06-06 01:10:34|It's a small plac...|Ue_7yUlkEbX4AhnYd...|
|GMrwDXRlAZU2zj5nH...|               0|2011-04-08 18:12:01|8 sandwiches, $24...|LltbT_fUMqZ-ZJP-v...|
|ALwAlxItASeEs2vYA...|               0|2015-05-20 20:17:38|Great ramen! Not ...|HHNBqfbDR8b1iq-QG...|
|d_L-rfS1vT3JMzgCU...|               0|2014-09-01 01:23:48|Cochinita Pibil w...|r0

In [9]:
# number of rows
print("Number of rows: {}.".format(dat.count()))

# number of unique businesses
print("Number of unique businesses: {}.".format(dat.select("business_id").distinct().count()))

# number of unique users
print("Number of unique users: {}.".format(dat.select("user_id").distinct().count()))

Number of rows: 1363162.
Number of unique businesses: 171010.
Number of unique users: 362556.


In [10]:
# identify power users
dat.groupby("user_id").count().orderBy(["count"], ascending=[0]).show(10)

+--------------------+-----+
|             user_id|count|
+--------------------+-----+
|                null|69231|
|mkbx55W8B8aPLgDqe...| 2600|
|CxDOIDnH8gp9KXzpB...| 1667|
|0tvCcnfJnSs55iB6m...| 1589|
|6ZC-0LfOAGwaFc5XP...| 1510|
|eZfHm0qI8A_HfvXSc...| 1324|
|O8eDScRAg6ae0l9Bc...| 1300|
|8DGFWco9VeBAxjqsu...| 1179|
|2EuPAGalYnP7eSxPg...| 1165|
|WJKocp9RE0KatUwh3...| 1111|
+--------------------+-----+
only showing top 10 rows



In [11]:
# identify power businesses
dat.groupby("business_id").count().orderBy(["count"], ascending=[0]).show(10)

+--------------------+-----+
|         business_id|count|
+--------------------+-----+
|FaHADZARwnY4yvlvp...| 3679|
|JmI9nslLD7KZqRr__...| 2494|
|DkYS3arLOhA8si5uU...| 1530|
|5LNZ67Yw9RD6nf4_U...| 1525|
|K7lWdNUhCbcnEvI0N...| 1434|
|hihud--QRriCYZw1z...| 1394|
|RESDUcs7fIiihp38-...| 1386|
|4JNXUYY8wbaaDmk3B...| 1185|
|yfxDa8RFOvJPQh0rN...| 1154|
|iCQpiavjjPzJ5_3gP...| 1145|
+--------------------+-----+
only showing top 10 rows



In [12]:
# filter tips with compliment > 0
print("{:.2f}% of observations have a compliment_count > 0.".format((dat.filter(dat.compliment_count > 0).count())/dat.count()*100))

1.37% of observations have a compliment_count > 0.


In [13]:
# max compliment count
dat.agg({"compliment_count": "max"}).collect()[0][0]

'zzcMR3izHZRiWvJI02GtIA'